<a href="https://colab.research.google.com/github/ferranfont/Introduccion_al_trading_algoritmico/blob/main/Quant_momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**QUANTITATIVE MOMENTUM SYSTEM**

In [5]:
import pandas as pd
import numpy as np
import requests
import matplotlib
import math

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Importamos todos los tickets del SP500
stocks = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [8]:
# Hacemos un API a un end point de momentum
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol='BA'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avg10Volume': 7962351,
 'avg30Volume': 10133928,
 'beta': 1.377717650518498,
 'companyName': 'Boeing Co.',
 'day200MovingAvg': 218.6,
 'day30ChangePercent': -0.024432092002296676,
 'day50MovingAvg': 216.28,
 'day5ChangePercent': -0.01614412709175465,
 'dividendYield': 0,
 'employees': 148008,
 'exDividendDate': '',
 'float': 0,
 'marketcap': 123429520941,
 'maxChangePercent': 2.24781577000412,
 'month1ChangePercent': -0.03805346733103195,
 'month3ChangePercent': -0.02470996046237167,
 'month6ChangePercent': -0.015437405887433881,
 'nextDividendDate': '',
 'nextEarningsDate': '',
 'peRatio': -14.566798092597688,
 'sharesOutstanding': 597228761,
 'ttmDividendRate': 0,
 'ttmEPS': -14.3,
 'week52change': -0.03977162707853909,
 'week52high': 280.56,
 'week52highSplitAdjustOnly': 278.78,
 'week52low': 185.9,
 'week52lowSplitAdjustOnly': 189.66,
 'year1ChangePercent': -0.04073659576726212,
 'year2ChangePercent': -0.3991635996284981,
 'year5ChangePercent': 0.3031450335731484,
 'ytdChangePerc

In [32]:
# Consultamos un campo en particular
data['month1ChangePercent']*100

-13.9268767883297

In [33]:
round(data['peRatio'],1)

8.1

In [34]:
my_colums = ['Ticker','Descripción', 'Rentab_1año','Rentab_6m','Rentab_1m','PER','n_acciones_a_comprar']
final_dataframe = pd.DataFrame(columns=my_colums)
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar


In [36]:
final_dataframe = final_dataframe.append(
                                        pd.Series([i, 
                                                   data['companyName'], 
                                                   round(data['year1ChangePercent'],2),
                                                   round(data['month6ChangePercent'],2),
                                                   round(data['month1ChangePercent'],2),
                                                   round(data['peRatio'],1),
                                                   'N/A'],  
                                                  index =my_colums), 
                                        ignore_index = True)
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,CAT,Caterpillar Inc.,-0.07,-0.05,-0.14,8.1,N/A


In [37]:
# Rellenamos el dataframe en cada una de sus filas (stocks tickers) con el valor que obtenemos de la API call
for i in stocks['Ticker'][:80]: #para no ralentizar el proceso lo haremos tan sólo para las primeros 80 símbolos [:80]
  api_url = f'https://sandbox.iexapis.com/stable/stock/{i}/stats?token=Tpk_059b97af715d417d9f49f50b51b1c448'
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
                                        pd.Series([i, 
                                                   data['companyName'], 
                                                   round(data['year1ChangePercent'],2),
                                                   round(data['month6ChangePercent'],2),
                                                   round(data['month1ChangePercent'],2),
                                                   round(data['peRatio'],1),
                                                   'N/A'], 
                                                  index =my_colums), 
                                        ignore_index = True)

In [38]:
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,CAT,Caterpillar Inc.,-0.07,-0.05,-0.14,8.1,N/A
1,A,Agilent Technologies Inc.,0.05,-0.22,-0.06,17.2,N/A
2,AAL,American Airlines Group Inc,-0.05,-0.05,0.03,-3.7,N/A
3,AAP,Advance Auto Parts Inc,0.35,0.06,-0.07,11.1,N/A
4,AAPL,Apple Inc,0.31,0.15,0.01,27.3,N/A
...,...,...,...,...,...,...,...
76,C,Citigroup Inc,0.00,-0.07,0.01,6.3,N/A
77,CAG,Conagra Brands Inc,0.09,0.09,0.01,16.1,N/A
78,CAH,"Cardinal Health, Inc.",0.08,0.08,0.04,27.1,N/A
79,CARR,Carrier Global Corp,0.18,-0.21,-0.10,11.9,N/A


In [29]:
# Vamos a retirar los que tienen bajo momentum con el método .sort_value()
final_dataframe.sort_values('rentabilidad_1_año', ascending=False, inplace=True)
final_dataframe.reset_index(inplace=True) #reseteamos el índice de nuevo
final_dataframe[:20]


,index,Ticker,Descripción,rentabilidad_1_año,PER,n_acciones_a_comprar
0,203,APA,APA Corporation,0.69,19.9,N/A
1,43,APA,APA Corporation,0.68,19.9,N/A
2,123,APA,APA Corporation,0.67,20.4,N/A
3,198,ANET,Arista Networks Inc,0.63,23.6,N/A
4,135,AZO,Autozone Inc.,0.62,17.8,N/A
5,38,ANET,Arista Networks Inc,0.62,23.2,N/A
6,118,ANET,Arista Networks Inc,0.62,23.9,N/A
7,215,AZO,Autozone Inc.,0.61,18.1,N/A
8,55,AZO,Autozone Inc.,0.60,17.8,N/A
9,40,ANTM,Anthem Inc,0.56,9.0,N/A
